# Clustering Part 3

In [1]:
# importing libraires
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import requests

In [4]:
# importing packages

#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import folium
print('Libraires are imported')

Libraires are imported


In [7]:
df_toronto = pd.read_csv("df_toronto_base.csv")
df_toronto.head(7)

,Unnamed: 0,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,0,M1T,Scarborough,Tam O'Shanter,43.781638,-79.304302
1,1,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
2,2,M6L,North York,Downsview,43.713756,-79.490074
3,3,M6J,West Toronto,"Little Portugal, Trinity",43.647927,-79.419750
4,4,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
5,5,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
6,6,M3K,North York,CFB Toronto,43.737473,-79.464763


In [8]:
# creating a map to toronto
toronto_latitude = 43.6532; toronto_longitude = -79.3832
map_toronto = folium.Map(location = [toronto_latitude, toronto_longitude], zoom_start = 10.7)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    

map_toronto

### create a newdataframe of scarborough and its neighbourhood

In [9]:
# @hiddel_cell
CLIENT_ID = '1VDJN4Q1FQWV3WCHJKYU0JNYVKYBKQFYXDRQ0IPWGC40ZNBX' # your Foursquare ID
CLIENT_SECRET = 'ZZJUO5GIFDDKB5PK1HQKJYHVQJGMZPQWKMT3Q2LCDVHY4Y3V' # your Foursquare Secret
VERSION = '20190724' # Foursquare API version

In [10]:
scarborough_data = df_toronto[df_toronto['Borough'] == 'Scarborough'].reset_index(drop=True)
scarborough_data.head(7)

,Unnamed: 0,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,0,M1T,Scarborough,Tam O'Shanter,43.781638,-79.304302
1,8,M1P,Scarborough,"Dorset Park, Scarborough Town Centre, Wexford ...",43.757410,-79.273304
2,20,M1G,Scarborough,Woburn,43.770992,-79.216917
3,21,M1N,Scarborough,Birch Cliff,43.692657,-79.264848
4,22,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
5,26,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,31,M1X,Scarborough,Upper Rouge,43.836125,-79.205636


### Create a map of Scarborough and its Neighbourhood

In [11]:
address_scar = 'Scarborough,Toronto'
latitude_scar = 43.773077
longitude_scar = -79.257774
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude_scar, longitude_scar))

The geograpical coordinate of Scarborough are 43.773077, -79.257774.


In [12]:
map_scarb = folium.Map(location=[latitude_scar, longitude_scar], zoom_start=12)

# add markers to map
for lat, lng, label in zip(scarborough_data['Latitude'], scarborough_data['Longitude'], scarborough_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_scarb)  
    
map_scarb

### getting top 100 locations of Tam O'Shanter from scarborough

In [13]:
neighborhood_latitude = scarborough_data.loc[0, 'Latitude'] # neighbourhood latitude value
neighborhood_longitude = scarborough_data.loc[0, 'Longitude'] # neighbourhood longitude value

neighborhood_name = scarborough_data.loc[0, 'Neighbourhood'] # neighbourhood name

print('Latitude and longitude values of "{}" are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of "Tam O'Shanter" are 43.7816375, -79.3043021.


In [14]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_scar, longitude_scar, VERSION, radius, LIMIT)

In [16]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d380f19a197930038630438'},
 'response': {'headerLocation': 'Scarborough City Centre',
  'headerFullLocation': 'Scarborough City Centre, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 43,
  'suggestedBounds': {'ne': {'lat': 43.7775770045, 'lng': -79.25155367954714},
   'sw': {'lat': 43.7685769955, 'lng': -79.26399432045285}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5085ec39e4b0b1ead2eb0818',
       'name': 'Disney Store',
       'location': {'address': '300 Borough Drive',
        'crossStreet': 'in Scarborough Town Centre',
        'lat': 43.775537,
        'lng': -79.256833,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.775537,
          'lng': -79.256833}],
        'distance

In [17]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [18]:
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

venues = results['response']['groups'][0]['items']  
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(10)

,name,categories,lat,lng
0,Disney Store,Toy / Game Store,43.775537,-79.256833
1,SEPHORA,Cosmetics Shop,43.775017,-79.258109
2,American Eagle Outfitters,Clothing Store,43.775908,-79.258352
3,Tommy Hilfiger Company Store,Clothing Store,43.776015,-79.257369
4,St. Andrews Fish & Chips,Fish & Chips Shop,43.771865,-79.252645
5,DAVIDsTEA,Tea Room,43.776613,-79.258516
6,Chipotle Mexican Grill,Mexican Restaurant,43.776410,-79.258069
7,Coliseum Scarborough Cinemas,Movie Theater,43.775995,-79.255649
8,Hot Topic,Clothing Store,43.775450,-79.257929
9,Tim Hortons,Coffee Shop,43.769936,-79.260368


In [19]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

43 venues were returned by Foursquare.


In [21]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### get venues for each neighbourhood in scarborough

In [22]:
scarborough_venues = getNearbyVenues(names=scarborough_data['Neighbourhood'],
                                   latitudes=scarborough_data['Latitude'],
                                   longitudes=scarborough_data['Longitude']
                                  )

Tam O'Shanter
Dorset Park, Scarborough Town Centre, Wexford Heights
Woburn
Birch Cliff
Clairlea, Golden Mile, Oakridge
Scarborough Village
Upper Rouge
Agincourt
Morningside, West Hill
Rouge, Malvern
Agincourt North, Milliken
Cliffcrest, Cliffside
Ionview, Kennedy Park
Maryvale, Wexford
Highland Creek, Rouge Hill, Port Union


In [23]:
scarborough_venues.head(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Tam O'Shanter,43.781638,-79.304302,The Royal Chinese Restaurant 避風塘小炒,43.780505,-79.298844,Chinese Restaurant
1,Tam O'Shanter,43.781638,-79.304302,Remezzo Italian Bistro,43.778649,-79.308264,Italian Restaurant
2,Tam O'Shanter,43.781638,-79.304302,Eight Noodles,43.778234,-79.308299,Noodle House
3,Tam O'Shanter,43.781638,-79.304302,Kub Khao,43.780438,-79.299837,Thai Restaurant
4,Tam O'Shanter,43.781638,-79.304302,TD Canada Trust,43.779169,-79.303617,Bank
5,Tam O'Shanter,43.781638,-79.304302,Little Caesars,43.780563,-79.298624,Pizza Place
6,Tam O'Shanter,43.781638,-79.304302,KFC,43.779440,-79.303371,Fast Food Restaurant
7,Tam O'Shanter,43.781638,-79.304302,Popeyes Louisiana Kitchen,43.780476,-79.298460,Fried Chicken Joint
8,Tam O'Shanter,43.781638,-79.304302,Gusto Pizza,43.783607,-79.298983,Pizza Place
9,Tam O'Shanter,43.781638,-79.304302,Rexall,43.780732,-79.298257,Pharmacy


In [24]:
scarborough_venues.tail(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
53,"Ionview, Kennedy Park",43.727929,-79.262029,Hakka No.1,43.727688,-79.266057,Chinese Restaurant
54,"Ionview, Kennedy Park",43.727929,-79.262029,Dollarama,43.726904,-79.265886,Discount Store
55,"Ionview, Kennedy Park",43.727929,-79.262029,Dollarama,43.726596,-79.266830,Discount Store
56,"Ionview, Kennedy Park",43.727929,-79.262029,Kennedy Station Passenger Pickup,43.732009,-79.264537,Bus Station
57,"Maryvale, Wexford",43.750071,-79.295849,Wexford Restaurant,43.746030,-79.293843,Breakfast Spot
58,"Maryvale, Wexford",43.750071,-79.295849,Frank's Smoke Shop,43.745890,-79.294940,Smoke Shop
59,"Maryvale, Wexford",43.750071,-79.295849,Sequoia Lounge,43.745645,-79.295737,Middle Eastern Restaurant
60,"Maryvale, Wexford",43.750071,-79.295849,Crown Pastries,43.746098,-79.293142,Bakery
61,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
62,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Affordable Toronto Movers,43.787919,-79.162977,Moving Target


In [25]:
scarborough_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Agincourt North, Milliken",3,3,3,3,3,3
Birch Cliff,4,4,4,4,4,4
"Clairlea, Golden Mile, Oakridge",7,7,7,7,7,7
"Cliffcrest, Cliffside",2,2,2,2,2,2
"Dorset Park, Scarborough Town Centre, Wexford Heights",7,7,7,7,7,7
"Highland Creek, Rouge Hill, Port Union",2,2,2,2,2,2
"Ionview, Kennedy Park",7,7,7,7,7,7
"Maryvale, Wexford",4,4,4,4,4,4


In [26]:
print('There are {} uniques categories.'.format(len(scarborough_venues['Venue Category'].unique())))

There are 44 uniques categories.


In [27]:
# one hot encoding
scarb_onehot = pd.get_dummies(scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scarb_onehot['Neighborhood'] = scarborough_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [scarb_onehot.columns[-1]] + list(scarb_onehot.columns[:-1])
scarb_onehot = scarb_onehot[fixed_columns]

scarb_onehot.head()

,Neighborhood,American Restaurant,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,Chinese Restaurant,...,Pizza Place,Playground,Print Shop,Rental Car Location,Sandwich Place,Skating Rink,Smoke Shop,Soccer Field,Thai Restaurant,Vietnamese Restaurant
0,Tam O'Shanter,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,Tam O'Shanter,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Tam O'Shanter,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Tam O'Shanter,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,Tam O'Shanter,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
scarb_onehot.shape

(63, 45)

In [29]:
scarb_grouped = scarb_onehot.groupby('Neighborhood').mean().reset_index()
scarb_grouped.head(7)

,Neighborhood,American Restaurant,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,Chinese Restaurant,...,Pizza Place,Playground,Print Shop,Rental Car Location,Sandwich Place,Skating Rink,Smoke Shop,Soccer Field,Thai Restaurant,Vietnamese Restaurant
0,Agincourt,0.0,0.000000,0.0,0.0,0.25,0.000000,0.000000,0.00,0.000000,...,0.0,0.000000,0.0,0.0,0.25,0.25,0.0,0.000000,0.0,0.000000
1,"Agincourt North, Milliken",0.0,0.000000,0.0,0.0,0.00,0.000000,0.000000,0.00,0.000000,...,0.0,0.333333,0.0,0.0,0.00,0.00,0.0,0.000000,0.0,0.000000
2,Birch Cliff,0.0,0.000000,0.0,0.0,0.00,0.000000,0.000000,0.25,0.000000,...,0.0,0.000000,0.0,0.0,0.00,0.25,0.0,0.000000,0.0,0.000000
3,"Clairlea, Golden Mile, Oakridge",0.0,0.285714,0.0,0.0,0.00,0.285714,0.142857,0.00,0.000000,...,0.0,0.000000,0.0,0.0,0.00,0.00,0.0,0.142857,0.0,0.000000
4,"Cliffcrest, Cliffside",0.5,0.000000,0.0,0.0,0.00,0.000000,0.000000,0.00,0.000000,...,0.0,0.000000,0.0,0.0,0.00,0.00,0.0,0.000000,0.0,0.000000
5,"Dorset Park, Scarborough Town Centre, Wexford ...",0.0,0.000000,0.0,0.0,0.00,0.000000,0.000000,0.00,0.142857,...,0.0,0.000000,0.0,0.0,0.00,0.00,0.0,0.000000,0.0,0.142857
6,"Highland Creek, Rouge Hill, Port Union",0.0,0.000000,0.0,0.5,0.00,0.000000,0.000000,0.00,0.000000,...,0.0,0.000000,0.0,0.0,0.00,0.00,0.0,0.000000,0.0,0.000000


### top 10 venues per neighbourhood

In [30]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [31]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scarb_grouped['Neighborhood']

for ind in np.arange(scarb_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarb_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Skating Rink,Sandwich Place,Lounge,Breakfast Spot,Vietnamese Restaurant,College Stadium,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
1,"Agincourt North, Milliken",Coffee Shop,Park,Playground,Indian Restaurant,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
2,Birch Cliff,College Stadium,General Entertainment,Skating Rink,Café,Vietnamese Restaurant,Indian Restaurant,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
3,"Clairlea, Golden Mile, Oakridge",Bakery,Bus Line,Soccer Field,Bus Station,Fast Food Restaurant,Vietnamese Restaurant,Convenience Store,Indian Restaurant,General Entertainment,Fried Chicken Joint
4,"Cliffcrest, Cliffside",American Restaurant,Motel,Italian Restaurant,Indian Restaurant,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
5,"Dorset Park, Scarborough Town Centre, Wexford ...",Indian Restaurant,Chinese Restaurant,Latin American Restaurant,Light Rail Station,Vietnamese Restaurant,Pet Store,Bus Line,Bakery,Bank,General Entertainment
6,"Highland Creek, Rouge Hill, Port Union",Bar,Moving Target,Vietnamese Restaurant,Convenience Store,Indian Restaurant,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
7,"Ionview, Kennedy Park",Discount Store,Department Store,Bus Station,Chinese Restaurant,Coffee Shop,Convenience Store,Vietnamese Restaurant,Intersection,Indian Restaurant,General Entertainment
8,"Maryvale, Wexford",Bakery,Smoke Shop,Breakfast Spot,Middle Eastern Restaurant,Vietnamese Restaurant,Convenience Store,Indian Restaurant,General Entertainment,Fried Chicken Joint,Fast Food Restaurant
9,"Morningside, West Hill",Breakfast Spot,Rental Car Location,Electronics Store,Medical Center,Pizza Place,Mexican Restaurant,Intersection,Vietnamese Restaurant,Convenience Store,General Entertainment


### k-means to cluster the neighbourhood into 5 clusters

In [33]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

scarb_data = scarborough_data.drop(14)
# set number of clusters
kclusters = 5

scarb_grouped_clustering = scarb_grouped.drop('Neighborhood', 1)


# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scarb_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 
#len(kmeans.labels_)#=16
#scarborough_data.shape

array([4, 1, 4, 4, 4, 4, 3, 1, 4, 4])

In [34]:
scarb_merged = scarb_data

# add clustering labels
scarb_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
scarb_merged = scarb_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

scarb_merged

,Unnamed: 0,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,M1T,Scarborough,Tam O'Shanter,43.781638,-79.304302,4,Pizza Place,Italian Restaurant,Thai Restaurant,Bank,Fried Chicken Joint,Fast Food Restaurant,Pharmacy,Chinese Restaurant,Noodle House,Convenience Store
1,8,M1P,Scarborough,"Dorset Park, Scarborough Town Centre, Wexford ...",43.757410,-79.273304,1,Indian Restaurant,Chinese Restaurant,Latin American Restaurant,Light Rail Station,Vietnamese Restaurant,Pet Store,Bus Line,Bakery,Bank,General Entertainment
2,20,M1G,Scarborough,Woburn,43.770992,-79.216917,4,Coffee Shop,Korean Restaurant,Convenience Store,Intersection,Indian Restaurant,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
3,21,M1N,Scarborough,Birch Cliff,43.692657,-79.264848,4,College Stadium,General Entertainment,Skating Rink,Café,Vietnamese Restaurant,Indian Restaurant,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
4,22,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577,4,Bakery,Bus Line,Soccer Field,Bus Station,Fast Food Restaurant,Vietnamese Restaurant,Convenience Store,Indian Restaurant,General Entertainment,Fried Chicken Joint
5,26,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,4,Playground,Vietnamese Restaurant,College Stadium,Indian Restaurant,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
6,31,M1X,Scarborough,Upper Rouge,43.836125,-79.205636,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,39,M1S,Scarborough,Agincourt,43.794200,-79.262029,1,Skating Rink,Sandwich Place,Lounge,Breakfast Spot,Vietnamese Restaurant,College Stadium,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
8,40,M1E,Scarborough,"Morningside, West Hill",43.763573,-79.188711,4,Breakfast Spot,Rental Car Location,Electronics Store,Medical Center,Pizza Place,Mexican Restaurant,Intersection,Vietnamese Restaurant,Convenience Store,General Entertainment
9,48,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,4,Fast Food Restaurant,Print Shop,Vietnamese Restaurant,College Stadium,Indian Restaurant,General Entertainment,Fried Chicken Joint,Electronics Store,Discount Store,Department Store


### Visualize the clusters in map

In [35]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location = [latitude_scar, longitude_scar], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(scarb_merged['Latitude'], scarb_merged['Longitude'], scarb_merged['Neighbourhood'], scarb_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine each of the 5 clusters

In [36]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 0, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,Postal Code,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,M1V,-79.284577,0,Coffee Shop,Park,Playground,Indian Restaurant,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store


In [37]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 1, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,Postal Code,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,M1P,-79.273304,1,Indian Restaurant,Chinese Restaurant,Latin American Restaurant,Light Rail Station,Vietnamese Restaurant,Pet Store,Bus Line,Bakery,Bank,General Entertainment
7,M1S,-79.262029,1,Skating Rink,Sandwich Place,Lounge,Breakfast Spot,Vietnamese Restaurant,College Stadium,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
13,M1R,-79.295849,1,Bakery,Smoke Shop,Breakfast Spot,Middle Eastern Restaurant,Vietnamese Restaurant,Convenience Store,Indian Restaurant,General Entertainment,Fried Chicken Joint,Fast Food Restaurant


In [38]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 2, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,Postal Code,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,M1M,-79.239476,2,American Restaurant,Motel,Italian Restaurant,Indian Restaurant,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store


In [39]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 3, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,Postal Code,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,M1X,-79.205636,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 4, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,Postal Code,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1T,-79.304302,4,Pizza Place,Italian Restaurant,Thai Restaurant,Bank,Fried Chicken Joint,Fast Food Restaurant,Pharmacy,Chinese Restaurant,Noodle House,Convenience Store
2,M1G,-79.216917,4,Coffee Shop,Korean Restaurant,Convenience Store,Intersection,Indian Restaurant,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
3,M1N,-79.264848,4,College Stadium,General Entertainment,Skating Rink,Café,Vietnamese Restaurant,Indian Restaurant,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
4,M1L,-79.284577,4,Bakery,Bus Line,Soccer Field,Bus Station,Fast Food Restaurant,Vietnamese Restaurant,Convenience Store,Indian Restaurant,General Entertainment,Fried Chicken Joint
5,M1J,-79.239476,4,Playground,Vietnamese Restaurant,College Stadium,Indian Restaurant,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
8,M1E,-79.188711,4,Breakfast Spot,Rental Car Location,Electronics Store,Medical Center,Pizza Place,Mexican Restaurant,Intersection,Vietnamese Restaurant,Convenience Store,General Entertainment
9,M1B,-79.194353,4,Fast Food Restaurant,Print Shop,Vietnamese Restaurant,College Stadium,Indian Restaurant,General Entertainment,Fried Chicken Joint,Electronics Store,Discount Store,Department Store
12,M1K,-79.262029,4,Discount Store,Department Store,Bus Station,Chinese Restaurant,Coffee Shop,Convenience Store,Vietnamese Restaurant,Intersection,Indian Restaurant,General Entertainment
